In [1]:
###############################################################################
# OPTIMIZATION & AI - LAB 07 - FERHAT Matthieu & LOVITON Pierre
###############################################################################

In [3]:
# System imports
import random

# Extra library imports
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [4]:
#User inputs
csvpath = 'choices-v1.csv'
csvpath2 = 'choices-v2.csv'

Assignment problems consist in associating agents to tasks in order to optimize an objective function. In fact, assigment problems are widely used, and may have crucial consequences in your everyday life, for example when a plateform assigns students to schools, according to their preferences, but also the grades they obtained, the available space in each school...


In this lab, we will try to assign projects to students so that they get as satisfied as possible! Many approaches can be used, but to conclude on the pure optimization part of this course, we choose to formulate and solve this problem using simulated annealing... and linear programming!

# I. Optimized assignment of 10 projects to 10 students

## a) Data loading

The AI & BI course was created this year! Because it was optional, only 10 students registered to it. To conclude the course, each student has to choose a project among 10 possibilities. At the end, each student will work on a different project. Because the teacher is nice, he asked the students their order of preferences to assign each of them to a different project. Let's have a look at these choices!

<font color='blue'> Question 1: load the 'choices-v1.csv' file using Pandas, and get automatically the number of students and projects. Display also the first five rows. Save the matrix containing only the rank values into the "rank_arr" variable. </font>

In [106]:
# Load the csv file
df = pd.read_csv('choices-v1.csv')

# Get automatically the number of students and projects
nb_students = df.shape[0]
nb_projects = df.shape[1]-1
print('{0} students and {1} projects'.format(nb_students,
                                             nb_projects))

# Create the rank array (size nb_students x nb_projects)
rank_arr = np.array(df.drop(['Student_id'],axis=1))

# Display the first five rows
df.head()

10 students and 10 projects


,Student_id,Project_01,Project_02,Project_03,Project_04,Project_05,Project_06,Project_07,Project_08,Project_09,Project_10
0,stud-01,10,6,7,4,2,8,3,1,9,5
1,stud-02,7,1,10,9,6,3,8,4,5,2
2,stud-03,9,6,8,7,4,2,1,10,3,5
3,stud-04,1,9,2,10,5,6,7,3,8,4
4,stud-05,8,3,9,2,6,1,4,10,7,5


We see that the first choice (with rank 1) of the first student is project 8, while his last choice (with rank 10) is the first project. We also see that each student had to rank all the projects, since each row contains all the integer values from 1 to 10.
Our assignment problem consists in assigning one project to one student. Doing a quick calculation, you should realize that brute-force is not applicable.

<font color='blue'> Question 2: what is the number of possible solutions? Write it as a function of the number of students (or projects).</font>

In [69]:
# Number of possible solutions
def possible_sol(nb_projects):
    facto = 1
    for i in range (1,nb_projects+1):
        facto = facto*i
    return facto

nb_sols = possible_sol(nb_projects)
print('{0} possible solutions'.format(nb_sols))

3628800 possible solutions


To optimize the assignment of one project to one student, we want to minimize the sum of the ranks of the selected projects. In fact, we can optimize this loss recognizing... a  **traveling salesman problem (TSP)**!

To that purpose, we create a graph with nb_students + nb_projects nodes, each node being associated with a student or a project. We have to create edges from each student node to each project node, associated with the rank. To obtain Hamiltonian cycles, we have to add edges from each project node to each student node, with cost $\epsilon$ (a small value compared to the minimal rank).
Thus, we can find the length of the Hamiltonian cycle starting from and ending to the node $S_1$ associated with the first project. The principle is illustrated in the given figure 'help.png'. The weighted adjacency matrix in this figure also illustrates that this is an asymmetric TSP...

We have now to compute this weighted adjacency matrix!

<font color='blue'> Question 3: compute the weighted adjacency matrix of our graph to formulate the assignment problem as a traveling salesman problem. Set $\epsilon$ to 0,1. </font>

In [376]:
'''# Fixed parameter
eps = 0.1

# Compute the weighted adjacency matrix
a = np.c_[np.zeros([nb_students,nb_students]), rank_arr]
b = np.c_[np.full((nb_projects, nb_projects), eps),np.zeros([nb_students,nb_students])] 

A_arr = np.r_[a,b]'''

'# Fixed parameter\neps = 0.1\n\n# Compute the weighted adjacency matrix\na = np.c_[np.zeros([nb_students,nb_students]), rank_arr]\nb = np.c_[np.full((nb_projects, nb_projects), eps),np.zeros([nb_students,nb_students])] \n\nA_arr = np.r_[a,b]'

In [177]:
# Fixed parameter
eps = 0.1

# Compute the weighted adjacency matrix
A_arr = np.zeros((nb_students + nb_projects,
                  nb_students + nb_projects))

A_arr[nb_students:nb_students + nb_projects, 0:nb_students] = eps
A_arr[0:0+rank_arr.shape[0], nb_students:nb_students+rank_arr.shape[1]] = rank_arr

We have now to compute an initial solution x0, which is a cycle. A cycle, as we saw during the course, is a sequence of adjacent nodes starting from and ending to the same node (associated with stud-01 in our case). Because of our formulation, a cycle must alternates nodes with index in [0, nb_students - 1] (student node) and nodes with index in [nb_students, nb_students + nb_projects - 1] (project node).

We choose a **nearest neighbours approach**. More precisely, we first select the node with index 0 (stud-01) and associate it with the project of minimal rank (Project_08). Thus, Project_08 cannot be associated to any other student. We then select the node with index 1 (stud-02) and associate it with the remaining project of minimal rank (Project_02). We continue and so on, until there is no couple of student/project to associate. Do not forget to loop to node 0 to obtain a cycle !

<font color='blue'> Question 4: compute the initial solution using the proposed nearest neighbours approach. </font>

In [181]:
'''# Initialize the solution using nearest neighbours
x0 = []
new_arr = A_arr
idx = []
lst= []
for i in range (nb_students):
    x0.append(i)
    lst = new_arr[i,nb_projects:]
    for ii in range (len(lst)):
        if np.min(lst) not in idx:
            x0.append(int((np.where(new_arr[i]==np.min(lst)))[0]))
            break
        else :
            lst.remove(np.min(lst))
x0.append(0)
# Display the initial solution
print(x0)'''

'# Initialize the solution using nearest neighbours\nx0 = []\nnew_arr = A_arr\nidx = []\nlst= []\nfor i in range (nb_students):\n    x0.append(i)\n    lst = new_arr[i,nb_projects:]\n    for ii in range (len(lst)):\n        if np.min(lst) not in idx:\n            x0.append(int((np.where(new_arr[i]==np.min(lst)))[0]))\n            break\n        else :\n            lst.remove(np.min(lst))\nx0.append(0)\n# Display the initial solution\nprint(x0)'

In [182]:
# Initialize the solution using nearest neighbours
init = 0
x0=[]
idxs=[]
for i in A_arr[0:10]:
    for j in idxs:
        i[j]=20
    idx = np.argmin(i[10:20])
    idxs.append(idx+10)
    x0.append(init)
    x0.append(idx+10)
    init+=1
    
x0.append(0)
    
print(x0)

[0, 17, 1, 11, 2, 16, 3, 10, 4, 15, 5, 19, 6, 18, 7, 13, 8, 14, 9, 12, 0]


You should have the following (here incomplete) list: [0, 17, 1, 11, ..., 0]

To compute the loss we want to minimize in our graph formulation, namely the length of the optimal cycle, you can write the following command:

In [11]:
n =20

In [12]:
np.sum(A_arr[x0[:n], x0[1:]]) 

25.000000000000004

To get the real loss, namely the sum of the rank of each selected project, we have to remove the artificially added costs $\epsilon$ , and the cost between the last selected project node and the first project node (added to make the loop):

In [13]:
np.sum(A_arr[x0[:n-1], x0[1:n]]) - eps * n * 0.5

23.900000000000002

Our initial solution is not so bad... but we can improve it, for example using simulated annealing!
We will use the same notion of neighborhood as in the previous lab. However, to ensure that we still can obtain a cycle, we have to permute either two student nodes or two project nodes. We choose to **permute two project nodes to define the neighbor solution**.

<font color='blue'> Question 5: complete the function below to perform simulated annealing.</font>

In [112]:
def sa(A_arr, x0, T0, tau, nb_its):
    """ Optimization using simulated annealing
        
    Parameters
    ----------
    A_arr: square array of size 
    (nb_projects + nb_students) x (nb_projects + nb_students)
        the weighted adjacency matrix
    x0: list/tuple of positive integers of size (nb_projects + nb_students)
        initial path
    T0: float
        temperature parameter
    tau: float
        temperature decrease hyperparameter
    nb_its: strictly positive integer
        number of iterations
        
    Returns
    -------
    x: array of positive integers of size (nb_projects + nb_students)
        vector of selected indexes after optimization
    losses: list of floats
        vector of loss values over the iterations
        
    Command:
     x, losses = sa(A_arr, x0, T0, tau, nb_its)
    """  
    
    # Number of projects + students
    n = len(x0) - 1
    
    # Initialize the loss vector
    losses = []
    
    # Initialize the temperature
    T = np.copy(T0)
    
    # Initialize the vector of solutions
    x = np.copy(x0)
    
    # Vector of node identifiers (looping to 0)
    node_ids = list(range(n))
    node_ids.append(0)
    


    # Iterative process
    for it in range(nb_its):

        
        # Random permutation of two projects' nodes
        a = random.sample(range(10,20),2)
        x2 = list(np.copy(x))
        x2[x2.index(a[0])]=a[1]
        x2[x2.index(a[1])]=a[0]

        # Compute the loss of the current solution
        # (length of the optimal cycle)
        loss_arr_1 = A_arr[x[:n-1], x[1:n]] 
        loss1 = np.sum(loss_arr_1[loss_arr_1 > eps])
        
        # Compute the loss of the neighbor solution 
        # (length of the candidate cycle)
        loss_arr_2 = A_arr[x2[:n-1], x2[1:n]] 
        loss2 = np.sum(loss_arr_2[loss_arr_2 > eps])
        
        # Perform the simulated annealing step
        # Compute the difference between the two solutions
        delta = loss2-loss1
        
        # Perform the simulated annealing step
        if delta <= 0:
            x =x2
            new_loss = loss2
        else:
            if random.random() <= np.exp(-delta/T):
                x =x2
                new_loss = loss2
            else:
                new_loss = loss1
         
        # Update the temperature hyperparameter        
        T = T * tau 
    
        # Update the loss vector
        losses.append(new_loss)
        
    return x, losses

<font color='blue'> Question 6: run simulated annealing 100 times starting from $x_0$, setting each time $\tau=0.9$, $T_0$=nb_projects (or nb_students) and 100 iterations. Knowing that the expected optimal loss value, *i.e.* the sum of the ranks of the selected projects, should be 17, what is your percentage of success? Comment.</font>

In [171]:
# Hyperparameter setting
T0 = nb_projects 
tau = 0.9
nb_its = 100


# Optimization using Simulated Annealing
x,losses = sa(A_arr, x0, T0, tau, nb_its)
# Display the optimized loss value
print("The optimized loss value is",losses[-1])

The optimized loss value is 17.0


In [174]:
final_losses = []

for i in range(100):
    x, losses = sa(A_arr, x0, T0, tau, nb_its)
    final_losses.append(int(losses[-1]))

ratio_17 = np.bincount(final_losses)[17] / len(final_losses)
print(f'Percentage of success: {ratio_17*100} %')

Percentage of success: 10.0 %


#### How many combinations are possible ? Compute the one giving the best mean rank over the selected projects. Until how many students can you use this method to get the best score in a reasonable time ?

# I. Optimized assignment of 10 projects to 30 students

The AI & BI course was so popular last year that this year, 30 students registered to this course. This is a success! However, the teacher chooses to let the number of projects inchanged, but to allow students to be working together. Thus, there are more students than projects. Let's have a look at the choices of the new students!

<font color='blue'> Question 7: load the 'choices-v2.csv' file using Pandas, and get automatically the number of students and projects. Display also the first five rows. Save the matrix containing only the rank values into the "rank_arr" variable. </font>

In [183]:
# Load the csv file
df = pd.read_csv('choices-v2.csv')

# Get automatically the number of students and projects
nb_students = df.shape[0]
nb_projects = df.shape[1]-1
print('{0} students and {1} projects'.format(nb_students, nb_projects))

# Create the rank array (size nb_students x nb_projects)
rank_arr = np.array(df.drop(['Student_id'],axis=1))


# Display the first five rows
df.head()

30 students and 10 projects


,Student_id,Project_01,Project_02,Project_03,Project_04,Project_05,Project_06,Project_07,Project_08,Project_09,Project_10
0,stud-01,3,4,7,10,6,1,8,2,5,9
1,stud-02,2,1,5,3,10,6,8,4,7,9
2,stud-03,4,3,6,1,9,2,5,8,7,10
3,stud-04,4,9,1,2,3,5,8,10,7,6
4,stud-05,3,6,4,9,7,1,2,5,8,10


Because there are 30 students and 10 projects, the teacher chooses to allow students to work in **group of maximum 3 students**.
This situation cannot be formulated as a TSP, since a project node could be visited up to 3 times.
However, we can notice that limiting a project to 3 students is a constraint... thus, we can use linear programming instead!
The problem to solve can be formulated as:

$\displaystyle \min_{x_{ij}} \sum_{i=1}^{n_s}\sum_{j=1}^{n_p} x_{ij}r_{ij}
\, s.t. \begin{cases}
\displaystyle \sum_{j=1}^{n_p} x_{ij} = 1 \forall i \in [1, n_s]\\
\displaystyle \sum_{i=1}^{n_s} x_{ij} \leq m_j \forall j \in [1, n_p]\\
0 \leq x_{ij} \leq 1 \forall i \in [1, n_s], j \in [1, n_p]
\end{cases}$

with $n_s$ the number of students, $n_p$ the number of projects, $m_j$ the maximum number of students allowed to work on project $j$, $r_{ij}$ the rank given by student $i$ to project $j$, and $x_{ij}$ the $n_s * n_p$ variables to optimize, representing the decision to assign student i to project j ($x_{ij}=1$ if student i get project j, $x_{ij}=0$ otherwise).
The 3 constraints are justified by the following reasons:
- one student can choose only one project
- each project $j$ is limited to $m_j$ students
- $x_{ij}$ represents the binary decision to assign or not student $i$ to project $j$

<font color='blue'> Question 8: complete the following code to solve our assignment problem using linear programming. Did we got no more than 3 students working on each project? Compute the sum of the ranks of the selected projects. </font>

In [374]:
###................................Completed...............................###

# Set the coefficients of the linear objective function to be minimized
c = rank_arr.flatten()


# Set the inequality constraint matrix
A_ub = np.zeros((nb_projects,nb_projects*nb_students))

for i in range(nb_projects):
    for j in range(nb_students):
        A_ub[i,j*10+i] = 1    

# Set the inequality constraint vector
b_ub =np.full((nb_projects), 3)#np.full((2,1),3)

# Set the equality constraint matrix
A_eq = np.zeros((nb_students,nb_projects*nb_students))

for i in range (nb_students):
    ii = i*10
    j=ii+10
    A_eq[i,ii:j] = 1

# Set the equality constraint vector
b_eq = np.full((nb_students), 1)#np.full((1, nb_students), 1)

#Set the bounds
bounds = [(0,1)]
'''for i in range (nb_projects):
    bounds.append((0,1))'''

# Perform the optimization using the simplex method
res = linprog(c,
              A_ub=A_ub, b_ub=b_ub,
              A_eq=A_eq, b_eq=b_eq, 
              method='revised simplex',
              bounds=bounds)

# Display the optimization result
print(res,'\n')

# Get the optimized vector of variables
x = res.x
x = np.reshape(x, (30,10))

# Compute the number of students assigned to each project
project_arr = np.count_nonzero(x == 1, axis=0)
for i in range (project_arr.shape[0]):
    print("Project",i,"has",project_arr[i],"Students")
    
# Display the sum of the ranks of the selected projects
loss_arr = rank_arr[x==1].sum()
print("\nThe sum of the ranks of the selected projects is :", loss_arr)

     con: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
     fun: 37.0
 message: 'Optimization terminated successfully.'
     nit: 423
   slack: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      